## 6. Classification
### 1. Merge one hot encoded genres to lyrics

In [23]:
import pandas as pd

df = pd.read_csv('../dataset/Lyrics_en_artists_clean_train.csv', index_col=['Band'])
df_val = pd.read_csv('../dataset/Lyrics_en_artists_clean_test.csv', index_col=['Band'])
df

,Lyrics,acid,acoustic,adult,age,album,alternative,americana,and,anthem,...,underground,urban,vapor,vintage,viral,vocal,wave,western,world,worship
Band,,,,,,,,,,,,,,,,,,,,,
Basia,And the river bank talks of the waters of Marc...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
L.A. Guns,Street fight\r\r\r\r\r\nBlack night\r\r\r\r\r\...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Omarion,"Fussin' and fightin', we back at it again\r\r\...",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
The Jackson 5,The eyes of love will watch you\r\r\r\r\r\nAs ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
arco,Distant lies - calling you across the years \r...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Oscar Peterson,"Birds do it, bees do it\r\r\r\r\r\nEven educat...",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
The Isley Brothers,"Do me wrong, do me right\r\r\r\r\r\nTell me li...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ella Fitzgerald,Come\r\r\r\r\r\nCuddle closer\r\r\r\r\r\nDon't...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Paul McCartney,"It's a tug of war\r\r\r\r\r\nNo no, your troub...",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Label exploration  

In [24]:
%matplotlib inline

y_train = df.drop(['Lyrics'], axis=1)
y_test = df_val.drop(['Lyrics'], axis=1)
test_lyrics = df_val['Lyrics']
del df
del df_val
#pd.scatter_matrix(df, alpha = 0.3, figsize = (14,8), diagonal = 'kde');

In [5]:
import seaborn as sns
sns.heatmap(y_train.corr())

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
from sklearn.decomposition import PCA
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import numpy as np

pca = PCA()
pca.fit(y_train)

# Dimension indexing
dimensions = dimensions = ['Dimension {}'.format(i) for i in range(1,len(pca.components_)+1)]

# PCA components
components = pd.DataFrame(np.round(pca.components_, 4), columns = list(y_train.keys()))
components.index = dimensions

# PCA explained variance
ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1)
variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance'])
variance_ratios.index = dimensions

# Create a bar plot visualization
fig, ax = plt.subplots(figsize = (14,8))

# Plot the feature weights as a function of the components
components.plot(ax = ax, kind = 'bar');
ax.set_ylabel("Feature Weights")
ax.set_xticklabels(dimensions, rotation=0)


# Display the explained variance ratios
for i, ev in enumerate(pca.explained_variance_ratio_):
    ax.text(i-0.40, ax.get_ylim()[1] + 0.05, "Explained Variance\n          %.4f"%(ev))

# Return a concatenated DataFrame
pd.concat([variance_ratios, components], axis = 1)

### 3. Doc2Vec Deep learning LSTM classifier

In [2]:
from gensim.models.doc2vec import Doc2Vec

doc2vec = Doc2Vec.load("../dataset/doc2vec")
doc_vectors = doc2vec.wv
print("Number of word vectors: {}".format(len(doc_vectors.vocab)))

C:\Users\juanp\Anaconda3\envs\mlnd\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Number of word vectors: 77790


In [25]:
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy
import en_core_web_sm

nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['s', 'chorus'])

# python -m spacy download en
nlp = en_core_web_sm.load(disable=['parser', 'ner'])
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
    
def transform_lyric(lyric):
    lyric = simple_preprocess(str(lyric), deacc=True)

    new_lyric = list()
    for word in lyric:
        if(word not in stop_words):
            new_lyric.append(word)
    lyric = new_lyric


    lemma_lyric = list()
    doc = nlp(" ".join(lyric)) 
    lemma_lyric = list()
    for token in doc:
        if(token.pos_ in allowed_postags):
            lemma_lyric.append(token.lemma_)
    lyric = lemma_lyric
    return lyric

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juanp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
import numpy as np
Text_INPUT_DIM = 100
train_size = y_train.shape[0] 
test_size = y_test.shape[0] 
text_train_arrays = np.zeros((train_size, Text_INPUT_DIM))
text_test_arrays = np.zeros((test_size, Text_INPUT_DIM))

for i in range(train_size):
    text_train_arrays[i] = doc2vec.docvecs[i]
    
for i in range(test_size):
    test_lyrics[i] = transform_lyric(test_lyrics[i])
    text_test_arrays[i] = doc2vec.infer_vector(test_lyrics[i])
#text_test_arrays = doc2vec.infer_vector(test_lyrics)
#del test_lyrics

In [27]:
from keras import backend as K
K.clear_session()

In [28]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

model_conv = Sequential()
model_conv.add(Embedding(input_dim=train_size, output_dim=100))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(64, 5, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(LSTM(100))
model_conv.add(Dense(y_train.shape[1], activation='sigmoid'))
model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
model_conv.summary()

estimator = model_conv.fit(text_train_arrays, y_train, validation_data=(text_test_arrays,y_test), epochs = 2, batch_size=1024)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         43338500  
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 64)          32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_1 (Dense)              (None, 222)               22422     
Total params: 43,458,986
Trainable params: 43,458,986
Non-trainable params: 0
________________________________________________________________

In [29]:
import pandas as pd

df = pd.read_csv('../dataset/Lyrics_en_artists_clean_train.csv', index_col=['Band'])
train_lyrics = df['Lyrics']
del df

In [30]:
#Bryce Fox
#Lucy
#genres: "indie poptimism","modern alternative rock","modern rock"

lyric = "So bring your vibe over here here here \
Throw it up in the air air air, oh \
I run my hands through your hair hair hair \
And give you that love cuz you're oh so rare \
You look better over here then take you do over there \
And I don't wanna share \
 \
You don't let a good thing go to waste \
So I took the lipstick off that face \
I don't wanna share, yeah \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
I'll fix you up something real real tall \
It's looking up, we should take it all off \
Yeah \
You remind me why the stars don't fall \
I swear I'll love you like your last name's Ball \
I'll take you to the lair if we make it past the stairs \
I don't wanna share, yeah \
 \
You don't let a good thing go to waste \
So I took the lipstick off that face \
I don't wanna share, yeah \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
 \
Said I'm just tryna get to know ya \
So bite your tongue before I bite it for ya \
Yeah \
Said I'm just tryna get to know ya \
So bite your tongue before I bite it for ya \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
I said not yet \
(Said I'm just tryna get to know ya) \
(So bite your tongue before I bite it for ya) \
She said she ain't Lucy \
I said not yet \
(Said I'm just tryna get to know ya) \
She said she ain't Lucy \
I said not yet" 

lyric = transform_lyric(lyric)
print(lyric)

['bring', 'vibe', 'throw', 'air', 'air', 'air', 'run', 'hand', 'hair', 'hair', 'hair', 'give', 'love', 'rare', 'look', 'better', 'take', 'wanna', 'share', 'let', 'good', 'thing', 'go', 'waste', 'take', 'lipstick', 'face', 'wanna', 'share', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'say', 'lucy', 'say', 'yet', 'say', 'lucy', 'fix', 'something', 'real', 'real', 'tall', 'look', 'take', 'remind', 'star', 'fall', 'swear', 'love', 'last', 'name', 'ball', 'take', 'lair', 'make', 'past', 'stair', 'wanna', 'share', 'let', 'good', 'thing', 'go', 'waste', 'take', 'lipstick', 'face', 'wanna', 'share', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'say', 'lucy', 'say', 'yet', 'say', 'lucy', 'say', 'tryna', 'get', 'know', 'bite', 'tongue', 'bite', 'say', 'tryna', 'get', 'know

In [31]:
vector = np.zeros((1, Text_INPUT_DIM))
vector[0] = doc2vec.infer_vector(lyric)

sims = doc2vec.docvecs.most_similar([vector[0]], topn=2)
for sim in sims:
    print(train_lyrics[sim[0]], sim[1], sim[0])

[Talking]
Yeah, pimps up hoes down, yeah 
Hustlas, playas, gangstas, gangstas, yeah
Fa shiggedel, shiggedy, uh, click shit down
What is it, check this out

[E-40]
Raised in the heart of the ghetto, dipping and dodging the metro
Pitching the ? and coke that's what I use to pedal
Heavy metal, a j-jack of all trays
Then you pimp, that's a hustler in many ways

Acting bad in the traffic, the hustler with the package
Serving that cha-cha, that yell, they all tragic magic
Plastic baggage, jelly jars in microwaves
Got to have it, backwoods and purple hayes

Mess around in my side of town, get clowned down
John, Jane Doe, lost and found
Everybody know this young player's about his business
Riches, chickens I pimp, but lickeness

My L.I.P.I folks be digging this
Devon, Pimpy Gear, Max Queezthis
Gorilla, Scarp Down, no conscience
In this occupation you can't be generous

Me and my fellas be bossing, dipping the pander and flossing
No matter how much it's costing we do this often cause we

[Choru

In [32]:
y_pred = model_conv.predict(vector)
idx = (-y_pred[0]).argsort()[:10]
dict(zip(y_train.columns.values[idx],y_pred[0][idx]))

{'rock': 0.28408453,
 'pop': 0.2541779,
 'jazz': 0.16478156,
 'christmas': 0.12639225,
 'country': 0.11700781,
 'standards': 0.114419326,
 'adult': 0.114407666,
 'alternative': 0.10623583,
 'folk': 0.10595578,
 'vocal': 0.09854613}

In [33]:
lyric = "I hear you talkin' shit, bro you think you're the heat \
Please bow down to defeat you're barely mince meat \
Stop with the street talk, and start to do the street, walk \
Lock yourself in and tell me this, how you gonna battle with this sick shit that I spit bitch \
See you still flappin' your jaws, prepare to be thrown in the ocean \
In the middle with jaws \
Or prefer to be served and severed with claws, knock on the doors Of absolute death you may \
But please be ready for the automatic failures you may make \
So take this and wait for another dismembering, remembering \
The tethering of your dream and how it was minced to pieces \
Believe in yourself please, to save you now you'll be needin' \
Help from Jesus \
You think I'm copy, pastin' please, tell me how you're actin' up \
Better be quite or Imma bout to bitch smack your mouth shut \
For good, yeah man talk the hood talk better go back and walk the hood walk you ain't had the taste of a real G \
Get back down to your level and prepare to be beat \
Writin' on your sheet, like you're deep please bitch your weak \
You ain't on my level or a level at all, take a vacation \
Up the damn wall and fall \
Humpty Dumpty, I must be rusty but you better knock the shit Down if you goin' against me \
I'm hard to beat, you're hardly cheese compared to these beats And this emcee \
You ready, alright, so come on down, I bet you to do it \
Let's see if you got the balls or if you're just truant \
To me, yeah let's keep an eye, on your ass as the heat intensifies \
Talk to me in a smart tone, prepare to meet your demise \
Throw you, a hundred million miles is what im finna do \
Call me Son Goku \
I already told you, what I'm gonna do, finna plow through \
Yeah man try to tackle a topic more intense then mine \
I always rap from the heart, make the fake rappers sigh \
But I really really don't give at all a flying fuck \
My chance of making this shit is the same as seein' a flyin' truck \
Or it could all depend on some serious luck \
Maybe I'll release a track that is heard my millions \
Make it big and to the top and make billions \
But that's a dream for another day, I'm here to take you down \
Don't care what you say \
Freestylin' this rap shit to the grave \
Rap battlin' bitches in my spare time for loose change \
Please, all of you fake ass rappers are plan \
Make like like a plane and take the next trip away \
Far away, from here I don't need to here your shit anymore \
I'm tired of it now, next place to be, will be a fuckin' morgue \
But let me just let you all know this, I'm sick of it, all of it \
So listen next time I write another track \
Listen to the facts I spit, and find it through my old wack rap crap \
I know I've said some shit, that know I demise \
But just look at my own and, your own life \
But this time, through my eyes \
I got a couple little bitches that need a little call out \
But I'll reside, got more and more, don't get me wrong \
Now get the fuck back home and call your mom \
Continue to rap please, just leave me alone \
I'm done hearing wack disses like this \
I'm sick of this, listening to this petty as shit \
Bitch"
lyric = transform_lyric(lyric)
print(lyric)

['hear', 'talkin', 'shit', 'bro', 'think', 'heat', 'bow', 'defeat', 'barely', 'mince', 'meat', 'stop', 'street', 'talk', 'start', 'street', 'walk', 'lock', 'tell', 'go', 'battle', 'sick', 'shit', 'spit', 'bitch', 'see', 'still', 'flappin', 'jaw', 'prepare', 'throw', 'ocean', 'middle', 'jaw', 'prefer', 'serve', 'sever', 'claws', 'knock', 'door', 'absolute', 'death', 'may', 'please', 'ready', 'automatic', 'failure', 'may', 'make', 'take', 'wait', 'dismembering', 'remember', 'tethering', 'dream', 'mince', 'piece', 'believe', 'save', 'needin', 'help', 'think', 'copy', 'pastin', 'tell', 'actin', 'better', 'quite', 'imma', 'bout', 'bitch', 'smack', 'mouth', 'shut', 'good', 'talk', 'hood', 'talk', 'better', 'go', 'back', 'walk', 'hood', 'walk', 'taste', 'real', 'get', 'back', 'level', 'prepare', 'beat', 'writin', 'sheet', 'deep', 'bitch', 'weak', 'level', 'level', 'take', 'vacation', 'damn', 'wall', 'fall', 'humpty', 'dumpty', 'must', 'rusty', 'better', 'knock', 'shit', 'go', 'hard', 'beat', 

In [34]:
vector = np.zeros((1, Text_INPUT_DIM))
vector[0] = doc2vec.infer_vector(lyric)

sims = doc2vec.docvecs.most_similar([vector[0]], topn=2)
for sim in sims:
    print(train_lyrics[sim[0]], sim[1], sim[0])

Girl why do you got that look on your face 
Time is now girl I don't want to waste 
You shocked my eye when you stepped in the place 
I want to get with that baby 
Now there's no way I can let
Let you get past me now baby 
I've scoped you girl now you're mine 
It's time to let  
Let you shine all night baby 
Love girl let me take you out 
Show you around my town 
Look at them starin' at you baby 
Tonight girl you deserve some wine 
Here's to you being so fly 
Let me just let you shine now lady 
Girl why do you got an empty glass in your hand 
You got your girls but do you have a man? 
I feel your vibe you look so good 
I can't stand to look your way baby 
Tip the cup back long legs bare breasted 
The type of body that got my mind manifested 
Lyrical potion took me at the glance when 
I see you dance 
Highlight from the light from the back 
I like the way you make me feel 
Got that sex appeal 
So swing it over this way tonight 
I want to make a long stay 0.6061241626739502 23851
One mor

In [35]:
y_pred = model_conv.predict(vector)
idx = (-y_pred[0]).argsort()[:10]
dict(zip(y_train.columns.values[idx],y_pred[0][idx]))

{'rock': 0.28408453,
 'pop': 0.2541779,
 'jazz': 0.16478156,
 'christmas': 0.12639225,
 'country': 0.11700781,
 'standards': 0.114419326,
 'adult': 0.114407666,
 'alternative': 0.10623583,
 'folk': 0.10595578,
 'vocal': 0.09854613}